# Panel chart - orthopedic procedures by cost and LOS

In [1]:
import pandas as pd
import random
import numpy as np
import altair as alt

### create dummy data

In [2]:
random.seed(0)
df = pd.DataFrame({'Procedure group': ['Knee replacement' for i in range(15)] + \
                                        ['Hip replacement' for i in range(15)] + \
                                        ['Cervical fusion' for i in range(15)] + \
                                        ['Lumbar fusion' for i in range(15)],
                  'Physician':random.sample(range(1000, 2000), 60),
                  'Cases':random.sample(range(5,100), 60),
                  'Avg cost': random.sample(range(2500, 5000), 15) + \
                                random.sample(range(2000, 7000), 15) + \
                                random.sample(range(5000, 12000), 15) + \
                                random.sample(range(6000, 15000), 15),
                  'LOS O/E':[random.uniform(0.5, 1.8) for i in range(60)]})
df['LOS O/E'] = np.round(df['LOS O/E'],2)

In [3]:
totalsDf = df[['Procedure group','Cases']].groupby('Procedure group').sum().reset_index()
nEquals = 'n = '
totalsDf['Header'] = totalsDf['Procedure group'] + '\n' + nEquals + totalsDf['Cases'].astype(str)

charttDf = df.merge(totalsDf[['Procedure group','Header']], on='Procedure group',how='left')
charttDf.sample(5)

,Procedure group,Physician,Cases,Avg cost,LOS O/E,Header
56,Lumbar fusion,1989,21,6354,1.58,Lumbar fusion\nn = 725
55,Lumbar fusion,1209,44,6596,1.47,Lumbar fusion\nn = 725
37,Cervical fusion,1150,99,5824,1.01,Cervical fusion\nn = 864
40,Cervical fusion,1747,72,10715,1.40,Cervical fusion\nn = 864
38,Cervical fusion,1317,93,11415,1.35,Cervical fusion\nn = 864


### Layer Altair charts

In [4]:
charttDf['Start1'] = 1; charttDf['Stop1'] = 1.25
charttDf['Start2'] = 1.25; charttDf['Stop2'] = 1.5

scatter=alt.Chart(charttDf).mark_circle(size=150,strokeWidth=.5,stroke='#696969').encode(
        x=alt.X("LOS O/E:Q",scale=alt.Scale(domain=(0.5, 1.5),nice=False, clamp=True),
                axis=alt.Axis(grid=False,title='LOS O/E',labelAngle=0,tickSize=0,format='.2f',
                                values=[.5,.75,1,1.25,1.5],
                                labelExpr="datum.label % 1.5 ? datum.label: '1.50+'")),
        y=alt.Y("Avg cost:Q",axis=alt.Axis(title='Average supply cost',tickCount=4,format='$,.0f')),
        color=alt.Color('Cases:Q',scale=alt.Scale(range=['#A6DBD2','#018c8a']),legend=None))

ruler = alt.Chart(charttDf).mark_rule(stroke="#ddd",strokeWidth=1).encode(x=alt.X('Start1:Q'))

area1 = alt.Chart(charttDf).mark_rect(color='#696969',opacity=0.05).encode(
    x=alt.X('min(Start1):Q',scale=alt.Scale(domain=(0.5, 1.5),nice=False, clamp=True)), 
    x2='min(Stop1):Q', y=alt.value(0), y2=alt.value(550))

area2 = alt.Chart(charttDf).mark_rect(color='#696969',opacity=0.1).encode(
    x=alt.X('min(Start2):Q',scale=alt.Scale(domain=(0.5, 1.5),nice=False, clamp=True)),
    x2='min(Stop2):Q',y=alt.value(0), y2=alt.value(550))

alt.layer(area1+area2+ruler+scatter
            ).properties(height=550,width=175
            ).facet(alt.Column('Header:N',sort=['Knee replacement', 'Hip replacement',
                                'Cervical fusion', 'Lumbar fusion'],title=None),spacing=30
            ).properties(title='Orthopedic physician performance by procedure group'
            ).configure(lineBreak='\n'
            ).configure_header(labelFontStyle='bold',labelColor='#6e6e6e',labelFont='Arial',
                               labelFontSize=14,labelPadding=12
            ).configure_view(strokeWidth=1
            ).configure_axis(labelColor='#6e6e6e',labelFont='Arial',labelFontSize=12,
                             titleFont='Arial',titleFontSize=12,titleColor='#6e6e6e'
            ).resolve_scale(color='independent'
            ).configure_title(fontSize=16,color='#6e6e6e',font='Arial',anchor='start',offset=25)

alt.FacetChart(...)

### Add a legend

<img src="..\assets\images\orthoLegend.png" width="300" height="200" />